# Enter Fisheries

In [1]:
from theano.sandbox import cuda
cuda.use('gpu1')


Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
WARNING (theano.sandbox.cuda): Ignoring call to use(1), GPU number 0 is already in use.


In [2]:
%cd /home/ubuntu/nbs/lesson4
%matplotlib inline
from __future__ import print_function, division
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

/home/ubuntu/nbs/lesson4


Using Theano backend.


In [3]:
batch_size = 64

## Setup batches 

To do:
1. create validation set
2. create sample set
2. mode to separate dirs for each set
3. finetune and train

In [4]:
from __future__ import division,print_function
import math
import cPickle as pickle
import os, json
from glob import glob
import numpy as np
from matplotlib import pyplot as plt

import PIL
from PIL import Image
#import cv2
from numpy.random import random, permutation, randn, normal, uniform
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom
from scipy.ndimage import imread
from sklearn.metrics import confusion_matrix
import bcolz
import itertools

In [5]:
%cd "/home/ubuntu/data/fisheries"

/home/ubuntu/data/fisheries


In [6]:
%cd train

/home/ubuntu/data/fisheries/train


### create validation set

In [7]:
%mkdir ../valid

mkdir: cannot create directory ‘../valid’: File exists


In [8]:
for d in glob('*'):
    print(d)
    os.mkdir('../valid/'+d)

DOL


OSError: [Errno 17] File exists: '../valid/DOL'

In [ ]:
g = glob('*/*.jpg')
shuf = np.random.permutation(g)
for i in range(400): os.rename(shuf[i], '../valid/'+shuf[i])

In [9]:
%ls -1 */*.jpg | cut -f1 -d"/" | sort | uniq -c

   1551 ALB
    183 BET
    105 DOL
     56 LAG
    408 NoF
    259 OTHER
    158 SHARK
    656 YFT
      1 ALB


In [10]:
 ls -1 ../valid/*/*.jpg | cut -f3 -d"/" | sort | uniq -c

    167 ALB
     17 BET
     12 DOL
     11 LAG
     57 NoF
     40 OTHER
     18 SHARK
     78 YFT


###  create sample

In [ ]:
from shutil import copyfile

In [ ]:
%mkdir ../sample
%mkdir ../sample/train
%mkdir ../sample/valid

In [ ]:
for d in glob('*'):
    print(d)
    os.mkdir('../sample/valid/'+d)
    os.mkdir('../sample/train/'+d)

In [ ]:
g = glob('*/*.jpg')
shuf = np.random.permutation(g)
for i in range(400): copyfile(shuf[i], '../sample/train/'+shuf[i])

In [11]:
 ls -1 ../sample/train/*/*.jpg | cut -f4 -d"/" | sort | uniq -c

    190 ALB
     13 BET
     12 DOL
      4 LAG
     50 NoF
     34 OTHER
     20 SHARK
     77 YFT


In [ ]:
%cd ../valid
g = glob('*/*.jpg')
shuf = np.random.permutation(g)
for i in range(100): copyfile(shuf[i], '../sample/valid/'+shuf[i])

In [12]:
 ls -1 ../sample/valid/*/*.jpg | cut -f4 -d"/" | sort | uniq -c

     42 ALB
      1 BET
      4 DOL
      3 LAG
     23 NoF
      8 OTHER
      1 SHARK
     18 YFT


### Create batches

In [14]:
path = '/home/ubuntu/data/fisheries/'
batches = get_batches(path+'train', batch_size=batch_size)

Found 3377 images belonging to 8 classes.


In [15]:
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

Found 400 images belonging to 8 classes.


In [16]:
test_batches = get_batches(path+'test', batch_size=batch_size*2, shuffle=False)

Found 1000 images belonging to 1 classes.


### Imagenet conv features

- Small data set similar to imagenet images (full color photos)
- Use pre-trained VGG weights
- Precompute the output of the last convolutional layer

In [17]:
vgg = Vgg16()
model = vgg.model
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]

In [18]:
conv_model = Sequential(conv_layers)

In [19]:
(val_classes, trn_classes, val_labels, trn_labels) = get_classes(path)

Found 3377 images belonging to 8 classes.
Found 400 images belonging to 8 classes.


In [20]:
conv_feat = conv_model.predict_generator(batches, batches.nb_sample)
conv_val_feat = conv_model.predict_generator(val_batches, val_batches.nb_sample)
conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)


OSError: [Errno 2] No such file or directory: '/home/ubuntu/data/fisheries/results/conv_val_feat.dat'

In [21]:
%mkdir /home/ubuntu/data/fisheries/results

In [22]:
save_array(path+'results/conv_val_feat.dat', conv_val_feat)
save_array(path+'results/conv_test_feat.dat', conv_test_feat)
save_array(path+'results/conv_feat.dat', conv_feat)

In [23]:
conv_feat = load_array(path+'results/conv_feat.dat')
conv_val_feat = load_array(path+'results/conv_val_feat.dat')
conv_val_feat.shape

(400, 512, 14, 14)

### Batchnorm dense layers on pretrained conv layers

In [24]:
def get_maxpool_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]